In [ ]:
## AGENDA
## - Page Rank

In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
#gph_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")

In [3]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[<Node id=-3 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>] relationships=[<Relationship id=-3 nodes=(<Node id=-3 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>, <Node id=-3 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>) type='LINKED_TO' properties={}>]>]

In [4]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET DATA

In [5]:
gph_conn.query("""
               CREATE (A:Node {name: "A"})
               CREATE (B:Node {name: "B"})
               CREATE (C:Node {name: "C"})
               CREATE (D:Node {name: "D"})
               
               CREATE (A)-[:LINKED_TO {weight: 1}]->(B)
               CREATE (A)-[:LINKED_TO {weight: 1}]->(D)
               CREATE (B)-[:LINKED_TO {weight: 1}]->(A)
               CREATE (C)-[:LINKED_TO {weight: 1}]->(B)
               CREATE (D)-[:LINKED_TO {weight: 1}]->(B)
               """)

[]

In [ ]:
## via GDS

In [15]:
gph_conn.query("""
               CALL gds.graph.drop("graph")
               """)
gph_conn.query("""
               CALL gds.graph.create("graph", "Node", {LINKED_TO: { relationship: "LINKED_TO", orientation: "UNDIRECTED", properties: {weight: {property: "weight",defaultValue: 1.0}}}})
               """)

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.drop`: Caused by: java.lang.IllegalArgumentException: Graph with name `graph` does not exist and can't be removed.}


[<Record graphName='graph' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {'weight': {'property': 'weight', 'defaultValue': 1.0, 'aggregation': 'DEFAULT'}}}} nodeCount=4 relationshipCount=10 createMillis=5>]

In [17]:
gph_conn.query("""
               CALL gds.alpha.closeness.stream("graph", {})
               YIELD nodeId, centrality AS score
               RETURN gds.util.asNode(nodeId).name AS nodeName, score
               ORDER BY score DESC
               """)

[<Record nodeName='B' score=1.0>,
 <Record nodeName='A' score=0.75>,
 <Record nodeName='D' score=0.75>,
 <Record nodeName='C' score=0.6>]

In [20]:
gph_conn.query("""
               CALL gds.alpha.betweenness.stream("graph", {})
               YIELD nodeId, centrality AS score
               RETURN gds.util.asNode(nodeId).name AS nodeName, score
               ORDER BY score DESC
               """)

[<Record nodeName='B' score=4.0>,
 <Record nodeName='A' score=0.0>,
 <Record nodeName='C' score=0.0>,
 <Record nodeName='D' score=0.0>]